In [2]:
import os
import sys
directory = os.getcwd()
parent_directory = directory.split("/")[:-3]
parent_directory = "/".join(parent_directory)
sys.path.append(parent_directory)
parent_directory

'/home/adminn/Desktop/Repositories/database-chat-assistant'

In [6]:
from pprint import pprint
from llama_index.core import Settings
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.llms.groq import Groq
import nest_asyncio
nest_asyncio.apply()
api_key="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"
llm = Groq(model="llama3-8b-8192",
    api_key=api_key,
    temperature=0.5
)
response = llm.complete("hi")
response.text

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

bacis llama single document agent experiment

In [7]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["LordOfTheRings.pdf"]).load_data()
len(documents)

1347

In [18]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embediing_model = HuggingFaceEmbedding(model_name='thenlper/gte-base')
Settings.llm = llm
Settings.embed_model=embediing_model

In [9]:
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)
pprint(nodes[12].metadata, indent=4)

{   'creation_date': '2024-05-16',
    'file_name': 'LordOfTheRings.pdf',
    'file_path': 'LordOfTheRings.pdf',
    'file_size': 13674248,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-05-14',
    'page_label': '13'}


Two indexes that llm will choose to use when quering the vDB

In [21]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import SummaryIndex, VectorStoreIndex, StorageContext
from app.utils.connections import chromadb_connection
def store_in_chroma(nodes):
    # creating or geting a connection from chromadb
    vector_collection = chromadb_connection("vec_idx")
    # creating a vector store
    chroma_vec_store = ChromaVectorStore(chroma_collection=vector_collection)
    # Creating storage_context
    vec_storage_context = StorageContext.from_defaults(vector_store=chroma_vec_store)
    # creating index for summery and per page vector in db
    vector_index = VectorStoreIndex(
        nodes,
        show_progress=True,
        storage_context=vec_storage_context
    )
    print(vector_collection.get(limit=5))
    return vector_index

index = store_in_chroma(nodes=nodes)
# v, s = load_indices()
index

Initializing query engine to perform search

In [13]:
from llama_index.core.tools import QueryEngineTool

# engine_summ = summary_index.as_query_engine(
#     response_mode="tree_summarize",
#     use_async=True
# )
engine_vect = index.as_query_engine()

# summary_tool = QueryEngineTool.from_defaults(
#     query_engine=engine_summ,
#     description="Used when queries are related to Summary of entire document."
# )
vector_tool = QueryEngineTool.from_defaults(
    query_engine=engine_vect,
    description="Used when queries require specific context instead of entire document."
)
engine_vect

Define a Router using the query engine tools created using indexes

In [14]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        # summary_tool,
        vector_tool
    ],
    verbose=True
)

In [20]:
from pprint import pprint
response = query_engine.query("what actually is durins bane")
pprint(str(response))
print(len(response.source_nodes))

Selecting query engine 0: The question is asking about the context of Durin's Bane, which is a specific concept in J.R.R. Tolkien's Middle-earth legendarium. The description 'Used when queries require specific context instead of entire document' suggests that this option is relevant to the question, as it implies that the context is important in understanding the concept..
"Durin's Bane."
2


Funtion tool calling basics

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import FunctionTool

vector_index = VectorStoreIndex(nodes, show_progress=True)
query_engine = vector_index.as_query_engine(similarity_top_k=2)
pprint(nodes[2].get_content(metadata_mode="all"), indent=4)